In [1]:
# DIRECTORY SET
import os
import sys
from pathlib import Path
base_dir=Path(os.getcwd()).parent.parent
os.chdir(base_dir)
print(os.getcwd())

d:\DestinyFolder\DestinyRecaps\DestinyRecapsApi\serverproject


In [2]:
import dotenv
dotenv.load_dotenv()

True

In [3]:
# DJANGO SETUP
import django
sys.path.append(os.path.abspath(''))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "serverproject.settings")
django.setup()

# Import async modules
import asyncio
from asgiref.sync import sync_to_async

# Import display modules
from IPython.display import display, Markdown

# Import other modules
import faiss

# import reloading
from importlib import reload

In [4]:
%load_ext autoreload
%autoreload 2

from destinyapp.models import StreamRecapData

from destinyapp.customlibrary import services
from destinyapp.customlibrary import utils

In [5]:
del services
from destinyapp.customlibrary import services
reload(services)

del utils
from destinyapp.customlibrary import utils
reload(utils)

<module 'destinyapp.customlibrary.utils' from 'd:\\DestinyFolder\\DestinyRecaps\\DestinyRecapsApi\\serverproject\\destinyapp\\customlibrary\\utils\\__init__.py'>

In [38]:
await services.bot_run_check()

d:\DestinyFolder\env\Lib\site-packages\django\db\models\fields\__init__.py:1659: RuntimeWarning: DateTimeField BotData.last_time_ran received a naive datetime (2024-07-05 23:45:45.691606) while time zone support is active.
  warnings.warn(


True

In [31]:
video_ids=await services.web_view_recent_stream_ids()

ALL YT IDS: ['IpXU3qNsAUk', 'AGmRpyc7cuQ', 'b4Pg1ZnAbBg', 'K_M_6YGdqNI', 'Cx2gfO2zY2c', 'H6rG1KUMnWU', '7dkRCmCQ38M', 'Mp8AJCmANhQ', '29ixeHeiLZI']
Driver closed


In [ ]:
https://youtu.be/y05Ho7OkvGA

In [36]:
video_id="y05Ho7OkvGA"

In [37]:
await services.download_video(video_id)

[youtube] Extracting URL: https://youtu.be/y05Ho7OkvGA
[youtube] y05Ho7OkvGA: Downloading webpage
[youtube] y05Ho7OkvGA: Downloading ios player API JSON
[youtube] y05Ho7OkvGA: Downloading m3u8 information
[info] y05Ho7OkvGA: Downloading 1 format(s): 251
[download] Destination: destinyapp\working_folder\working_audio\raw.webm
[download] 100% of    3.00MiB in 00:00:00 at 13.96MiB/s  
MoviePy - Writing audio in destinyapp/working_folder/working_audio/merged_audio.mp3


MoviePy - Done.
download thread finished
download thread closed


In [38]:
raw_transcript=await services.generate_assembly_transcript()

Starting assembly transcription thread
Finished assembly transcription thread


In [39]:
transcript, linked_transcript=await services.process_raw_transcript(raw_transcript, video_id)

{'text': 'looks', 'start': 186945, 'end': 187361, 'confidence': 0.97326, 'speaker': 'B', 'channel': None}
560
561
Destiny doesn't start
Finished diarization cutoff


In [37]:
vectordb, text_chunks=await services.VectorDbAndTextChunksGenerator.generate_basic_vectordb_and_chunks(video_id, transcript)

Number of chunks:  19
Finished generating embeddings
Finished making vector db
Saved vector db to: destinyapp/vector_dbs/B8Wb8arDcyQ.index


In [26]:
segments_and_summaries=await services.SummarizedSegmentGenerator.generate_summarized_segments(transcript)

0 0 0
Total cost:  0.014577


In [28]:
recap = await services.RecapGenerator.generate_recap(segments_and_summaries)

Recap generation:  0.010872


In [30]:
recap_hook=await services.RecapGenerator.generate_recap_hook(recap)

Recap hook generation cost:  0.004836


In [32]:
full_title=await services.get_video_metadata(video_id)

[youtube] Extracting URL: https://www.youtube.com/watch?v=B8Wb8arDcyQ
[youtube] B8Wb8arDcyQ: Downloading webpage
[youtube] B8Wb8arDcyQ: Downloading ios player API JSON
[youtube] B8Wb8arDcyQ: Downloading m3u8 information


In [31]:
finalized_recap=recap_hook+"\n"+recap+"\n\nDISCLAIMER: This is all AI generated and there are frequent errors."

In [36]:
text_chunks

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000001E114C608D0> >

In [38]:
# save everything into StreamRecapData object
stream_recap_data=StreamRecapData(video_id=video_id, video_characteristics={"title":full_title}, raw_transcript_data=raw_transcript, transcript=transcript, linked_transcript=linked_transcript, text_chunks=text_chunks, summarized_chunks=segments_and_summaries, recap=finalized_recap)

# async save
await sync_to_async(stream_recap_data.save)()

In [16]:
stream_recap_data=await utils.get_recap_data(video_id)

In [41]:
discord_recaps_to_send=await services.DiscordMessageHandler.compile_discord_messages([video_id])

In [42]:
await services.DiscordMessageHandler.send_discord_recaps(discord_recaps_to_send)

Discord logged in as DestinySummarizer#3997
Recaps
Voice Channels
general
General
recaps
rules
test
Server Guide
moderator-only
Discussion
discussion
discussion
destiny-clips
forum-discussion
General-2
General-3
memes
programming-discussion
landing-page
about
General
Development Discussion
features
recaps-testing
Sending Recap video_id:  B8Wb8arDcyQ  to channel:  recaps-testing
Sending chunks:  2


Send and client closed


# START OF DEBUGGING

In [41]:
recap_data=await utils.get_recap_data("SPaeA8shXFg")
transcript, linked_transcript=await services.video_and_transcript.process_raw_transcript(recap_data.raw_transcript_data, "SPaeA8shXFg")

{'text': 'looks', 'start': 186955, 'end': 187331, 'confidence': 0.9519, 'speaker': 'A', 'channel': None}
228
229
Destiny doesn't start
Finished diarization cutoff


In [42]:
transcript

"B: I recently took a JavaScript quiz and I myself did not get them right. So let's see this new quiz here by an ex fang developer. I am currently a fang developer at this moment.\n\nC: You, you sent me this JavaScript quiz and said it's absolutely impossible to get all the questions right. So let's try it out.\n\nB: See what, I already know the answer to this one. Man, this is the classic octal. This is the classic octal bamboozle going on right here. Okay, so I don't, I think this is 18 -15 and octal, which is 13, this is five is my guess.\n\nC: What we can get correct. And if we can learn something new along the way as well. So the first question, do you know what will be the output of this code console log? Zero one eight minus zero one five. Okay, so I believe if you start with a zero, the number becomes base eight. What is that? I guess octal is probably what that's called. Let's go octn number one eight. But the digits that are valid in octal I think are just zero through seven,

In [32]:
# DELETE A VIDEO ID
delete_video_id="SPaeA8shXFg"
await utils.delete_stream_recap_data(delete_video_id)

In [46]:
# DELETE A VIDEO ID
delete_video_id="Gej2eHRwlM0"

delete_obj=await sync_to_async(StreamRecapData.objects.filter)(video_id=delete_video_id)

await sync_to_async(delete_obj.delete)()

(1, {'destinyapp.StreamRecapData': 1})

In [ ]:
# from destinyapp.customlibrary import ServerAiFunctions as saf
# from destinyapp.customlibrary import controls as cl
del sys.modules['destinyapp.customlibrary.ServerAiFunctions']
del sys.modules['destinyapp.customlibrary.controls']
del sys.modules['destinyapp.customlibrary.processing']
import destinyapp.customlibrary.ServerAiFunctions as saf
import serverproject.destinyapp.customlibrary.controller as cl
import destinyapp.customlibrary.processing as ps
reload(saf)
reload(cl)
reload(ps)


In [ ]:
filled_recap_data=await cl.get_all_recaps()

In [ ]:
filled_recap_data[1]#["recap"]

In [ ]:
transcript_model_data=await ps.grab_transcript_data('2O8rCcpFswk')

In [ ]:
discord_message={"recap":transcript_model_data.recap,"yt_id":transcript_model_data.video_id, "title":transcript_model_data.video_characteristics.get("title","")}

In [ ]:
await ps.discord_message_handler.send_discord_recaps([discord_message])

In [ ]:
discord_recaps_to_send=await ps.discord_message_handler.compile_discord_messages(['2O8rCcpFswk'])

In [ ]:
discord_recaps_to_send

In [ ]:
await ps.discord_message_handler.send_discord_recaps(discord_recaps_to_send)

In [ ]:
discord_recaps_to_send